# 集成学习  

集成学习是一种机器学习范式，通过训练多个模型来解决同一问题。与尝试从训练数据中学习一个假设的普通机器学习方法相反，集成方法尝试构建一组假设并将其组合使用。下面将使用决策树以及其集成版本对经典数据集 Mnist 建模，观察不同集成方法的差异

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier

## 构建数据集

此次使用的 Mnist 数据集并非原始的格式，为了更加方便的适配本次训练，将原始数据集中 28 * 28 的图片进行了 [flatten](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) 操作，变成了 784 个特征，下方 DataFrame 中的列：1x1, 1x2, ..., 28x28，表示图片中第 *i* 行、第 *j* 列的像素值，由于是灰度图，所以像素值只有 0 和 1

In [2]:
train_df = df = pd.read_csv('/data/course_data/ensemble/mnist_train.csv')
train_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# 查看训练数据信息:，有无 NaN，共有多少条数据 ...
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB


In [3]:
# 查看测试数据信息:，有无 NaN，共有多少条数据 ...
test_df = df = pd.read_csv('/data/course_data/ensemble/mnist_test.csv')
test_df.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 59.9 MB


In [4]:
test_df.iloc[:,0]

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int64

In [5]:
# 构建训练、测试数据
X_train = train_df.iloc[:,1:]
y_train = train_df.iloc[:,0]
X_test = test_df.iloc[:,1:]
y_test = test_df.iloc[:,0]

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((60000, 784), (60000,)), ((10000, 784), (10000,)))

## 决策树

首先训练一个简单的决策树看看表现如何

In [16]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [17]:
dtc.score(X_train, y_train)

1.0

In [18]:
dtc.score(X_test, y_test)

0.8787

> 从上面的训练集准确度和测试集的准确度来看，默认参数的决策树已经发生了过拟合，下面对 sklearn 提供的决策树 api 中的参数进行适当的调节，看看会有什么变化

In [19]:
dtc = DecisionTreeClassifier(min_samples_leaf=8)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [20]:
dtc.score(X_train, y_train), dtc.score(X_test, y_test)

(0.9311666666666667, 0.8791)

> 从上面的结果看出，通过对参数 `min_samples_leaf` 的调整，过拟合的情况已经有所缓解。`min_samples_leaf` 的含义是决策树中叶子结点所包含的最少的样本数量，通过增大这个参数，可以让决策树在训练的时候无法去捕捉训练数据任何一个细微的特征，导致对训练数据过拟合；叶子结点的样本数量大，也可以起到一定的投票作用，增强模型的泛化性能。可以尝试继续增大该参数的值，试着找到最佳的参数。除了这个参数还可以尝试调节 `min_samples_split`、`max_features` 等参数。

In [21]:
# 尝试调节其他参数，看看决策树在测试集上的表现
dtc = DecisionTreeClassifier(max_depth = 20, min_samples_leaf=6)
dtc.fit(X_train, y_train)
dtc.score(X_train, y_train), dtc.score(X_test, y_test)

(0.94225, 0.8826)

## 随机森林

看看决策树的 bagging 版本 随机森林的表现如何吧！

In [22]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
rfc.score(X_train, y_train)

0.9991

In [24]:
rfc.score(X_test, y_test)

0.9468

> 不愧是集成版本，基本在默认参数下就达到了较好的性能，测试集准确度高出普通决策树 7 个百分点左右，但是对比训练、测试结果可以发现，仍然存在一定的过拟合情况，下面调整一些参数试试

In [25]:
rfc = RandomForestClassifier(n_estimators=20)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
rfc.score(X_train, y_train), rfc.score(X_test, y_test)

(0.9999, 0.9576)

> 在增大了参数 `n_estimators` 之后，测试集准确度提升了 1 个百分点左右，这个参数的含义是同时训练 20 个决策树，最后对结果进行集成，这个参数的增加可以简单的认为是投票的人数增加了，那么最后的结果必然也会更加鲁棒。可以尝试继续增大这个参数，或者调节其他参数如 `max_samples`，适当少于全部训练数据量，可以增加不同子模型之间的差异，进一步提升泛化性能。同还可以调节基学习器(决策树)的参数。参数含义具体见 [sklearn 文档](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

## GBDT

再来对比一下决策树的 boosting 版本 GBDT 的表现吧！

In [27]:
gbc = GradientBoostingClassifier(n_estimators=10)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [28]:
gbc.score(X_train, y_train)

0.8423

In [29]:
gbc.score(X_test, y_test)

0.846

> 观察以上结果，虽然模型没有过拟合，但是可能出现了欠拟合，训练集的准确度远低于普通决策数和随机森林的准确度，此时最简单的方法就是增大 `n_estimators` 参数的值，也就是增加子学习器的数量，从 GBDT 的原理出发，增加子学习器的数量也就是对标签的参差进行更加精细化的拟合，从而解决欠拟合。

In [30]:
gbc = GradientBoostingClassifier(n_estimators=30)
# GBDT 决策树调节参数后在测试集上的表现
gbc.fit(X_train, y_train)
gbc.score(X_train, y_train), gbc.score(X_test, y_test)

(0.9074333333333333, 0.9057)

> 意料之中，表现有较大提升，并且训练集的指标和测试集的基本保持一致，没有出现过拟合的情况，所以应该还可以继续尝试提高这个参数。一般在未出现过拟合的情况下，我们只需要考虑继续提升模型的复杂度就好，这是提升表现最快的方法。当模型复杂度提高到出现过拟合的情况，我们再考虑使用一些降低过拟合的方法。

## Bagging

前述的随机森林和 GBDT 都是以决策树为基学习器的集成学习算法，但是要注意的是集成学习并不是决策树的专属，任何其他的学习器都可以作为集成学习的基学习器，比如逻辑回归、支持向量机。


Bagging 是“bootstrap aggregating”的缩写。这是一个元算法，它从初始数据集中获取 M 个子样本（有放回），并在这些子样本上训练预测模型。最终模型是通过平均所有子模型获得的，通常会产生更好的结果。该技术的主要优点是它结合了正则化，你所需要做的就是为基学习器选择良好的参数。

下面使用 sklearn 提供的通用 api 构造集成学习算法

In [35]:
# 仍然以决策树作为基学习器
bgc = BaggingClassifier(DecisionTreeClassifier(), max_samples=0.5, max_features=1.0, n_estimators=20)
bgc.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [36]:
bgc.score(X_train, y_train)

0.9938

In [37]:
bgc.score(X_test, y_test)

0.9492

In [8]:
# 以逻辑回归作为基学习器的 bagging 算法
bgc = BaggingClassifier(LogisticRegression(solver='lbfgs', max_iter=500), max_samples=0.5, max_features=1.0, n_estimators=20)

In [ ]:
bgc.fit(X_train, y_train)
bgc.score(X_train, y_train), bgc.score(X_test, y_test)

In [6]:
lr_model = LogisticRegression(solver='lbfgs', max_iter=500)

In [7]:
lr_model.fit(X_train, y_train)
lr_model.score(X_train, y_train), lr_model.score(X_test, y_test)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(0.9405166666666667, 0.9219)

> 上面我们成功地使用了逻辑回归作为基学习器完成了集成学习。你们可以自己尝试仅使用逻辑回归进行训练，对比单模型的表现和 bagging 版本的逻辑回归

## Boosting

boosting 是指能够将弱学习器转换为强学习器的一系列算法。 boosting 的主要原理是将一系列弱学习器（仅比随机猜测好）。对于那些在训练前期而被错误分类的样本，boosting 算法会给予更大的重视。然后通过加权多数投票（分类）或加权和（回归）组合预测，以产生最终预测。

In [14]:
# 实现以决策树为基学习器的 boosting 算法
adc = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=10, learning_rate=0.01)
adc.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                       

In [13]:
adc.score(X_train, y_train), adc.score(X_test, y_test)

(1.0, 0.8765)

In [7]:
adc = AdaBoostClassifier(LogisticRegression(), n_estimators=10, learning_rate=0.01)
adc.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=LogisticRegression(C=1.0, class_weight=None,
                                                     dual=False,
                                                     fit_intercept=True,
                                                     intercept_scaling=1,
                                                     l1_ratio=None,
                                                     max_iter=100,
                                                     multi_class='auto',
                                                     n_jobs=None, penalty='l2',
                                                     random_state=None,
                                                     solver='lbfgs', tol=0.0001,
                                                     verbose=0,
                                                     warm_start=False),
                   learning_rate=0.01, n_estimators=10, random_state=None)

In [9]:
adc.score(X_train, y_train), adc.score(X_test, y_test)

(0.9316666666666666, 0.9231)

> 对比决策树和逻辑回归的 boosting 集成版本可以发现，逻辑回归相对来说泛化能力更好，决策树更容易过拟合

In [17]:
adc = AdaBoostClassifier(DecisionTreeClassifier(min_samples_leaf=8), n_estimators=10, learning_rate=0.01)
adc.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=8,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                       

In [18]:
adc.score(X_train, y_train), adc.score(X_test, y_test)

(0.9978333333333333, 0.9507)

> 事实上过拟合并不是一件坏事，如果你的模型都无法过拟合的话说明它无法很好的拟合训练数据，所以决策树在一开始过拟合的很厉害这也说明他的潜力，可以看到上面经过参数的调节，决策树的 boosting 版本轻松地超过了逻辑回归的 boosting 版本